In [1]:
from pathlib import Path
from itertools import product
import numpy as np
import pandas as pd


In [2]:
# Directories
bids_dir = Path('/projects/osl/xiy/data/210817SAGE2Data/bids_data')
deriv_dir = Path('/projects/osl/xiy/data/210817SAGE2Data/derivatives')
code_dir = Path('/projects/osl/xiy/scripts')
log_dir = Path('/projects/osl/xiy/log')
submit_dir = Path('/projects/osl/xiy/submit')
work_dir = submit_dir / 'works'

excluded = ['001', '023', '050', '093']
subj_info = pd.read_csv(bids_dir.joinpath('participants.tsv'), delimiter='\t')
subj_info['included'] = 'yes'
subj_info.loc[subj_info.participant_id.isin(excluded), 'included'] = 'no'
subj_info.to_csv(bids_dir.joinpath('participants.tsv'), sep='\t', index=False)

In [10]:
# the following IDs excluded because their fMRIPrep have been run separately
# excluded = ['sub-001']
subj_info = pd.read_csv(bids_dir.joinpath('participants.tsv'), delimiter='\t')
subj_info['included'] = 'yes'
# subj_info.loc[subj_info.participant_id.isin(excluded), 'included'] = 'no'
subj_info.to_csv(bids_dir.joinpath('participants.tsv'), sep='\t', index=False)

In [11]:
# Get participants list
# subj_info = pd.read_csv(bids_dir.joinpath('participants.tsv'), delimiter='\t')
subj_list = [
    subj_id.replace('sub-', '')
    for subj_id in subj_info.loc[subj_info.included == 'yes', 'participant_id'].tolist()
]

In [6]:
# to create a txt file with each line detailing a job request
out_file = submit_dir / 'preproc_fmriprep_slicetiming.txt'
op_string = ''
for subj_id in subj_list:
    op_string += (
        f'singularity run -e --bind /projects/osl/xiy:/projects/osl/xiy /projects/osl/xiy/images/fmriprep-20.2.3.simg '
        f'{bids_dir} {deriv_dir} participant '
        f'--participant_label {subj_id} '
        '--output-space MNI152NLin2009cAsym MNI152NLin6Asym '
        f'-w {work_dir} --fs-license-file /projects/osl/xiy/license.txt --resource-monitor '
        '--notrack --stop-on-first-crash --error-on-aroma-warnings --ignore slicetiming --use-aroma\n')
    out_file.write_text(op_string)

In [6]:
# to create a smooth and scale job request
nii_info = pd.read_csv(deriv_dir.joinpath('fmriprep/smoothlist'))
op_string = ''
for subj_id in subj_list:
    op_string += (
        f'module load afni '
        f'{bids_dir} {deriv_dir} participant '
        f'--participant_label {subj_id} '
        '--output-space MNI152NLin2009cAsym MNI152NLin6Asym ')
    out_file.write_text(op_string)

    sub-001/ses-001/func/sub-001_ses-001_task-reward_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
0    sub-001/ses-001/func/sub-001_ses-001_task-rewa...                                                 
1    sub-001/ses-001/func/sub-001_ses-001_task-rewa...                                                 
2    sub-001/ses-002/func/sub-001_ses-002_task-rewa...                                                 
3    sub-001/ses-002/func/sub-001_ses-002_task-rewa...                                                 
4    sub-001/ses-002/func/sub-001_ses-002_task-rewa...                                                 
..                                                 ...                                                 
123  sub-810/ses-001/func/sub-810_ses-001_task-rewa...                                                 
124  sub-810/ses-001/func/sub-810_ses-001_task-rewa...                                                 
125  sub-810/ses-002/func/sub-810_ses-002_task-rewa...          

In [23]:
session_list = ['01', '02']
task_list = ['reward', 'reward']
result = zip(session_list, task_list)
print(set(result))

{('01', 'reward'), ('02', 'reward')}


In [29]:
#Flavor1: T1w space; Smooth 4mm; only reward task is smoothed & scaled. Resting task has a TR of .78 instead of 2
workflow = code_dir / 'preprocess' / 'workflow_PostFMRIPREP.py'
data_dir = Path('/projects/osl/xiy/data/210817SAGE2Data')

session_list = ['001', '002']
task_list = ['reward', 'reward']
# run_list = ['01 02 03 04 05 06', '01 02 03 04 05 06']
bold_space = 'MNI152NLin2009cAsym'

#######need to find a way to create a list such that 02 session won't be created for participants with only one visit
####best approach: original tsv change into subj_id, subj_id(not_unique), session, don't use the generic zip function
output_dir = deriv_dir.joinpath('preproc_variant_smooth4')
out_file = submit_dir.joinpath(f'preproc_post-fmriprep_space-{bold_space}_variant_smooth4.txt')

op_string = ''
for bold_id, subj_id in product(zip(session_list, task_list), subj_list):

    session_id = bold_id[0]
    task_id = bold_id[1]
    work_dir = bids_dir.joinpath('works', f'ses-{session_id}', (f'preproc_post-fmriprep_ses-{session_id}_task-{task_id}_'
                                           f'space-{bold_space}_variant-smooth4'))

    op_string += (
        f'python3 {workflow} '
        f'{data_dir} {output_dir} participant '
        f'--participant_label {subj_id} --session-id {session_id} --task-id {task_id} '
        f'--bold-space {bold_space} --brainmask '
        f'fmriprep '
        '--repetition-time 2.0 --smooth-fwhm 4 '
        f'--n_procs 6 --work-dir {work_dir}\n')
    out_file.write_text(op_string)


In [15]:
#Flavor2: T1w space; Smooth 6mm
workflow = code_dir / 'preprocess' / 'workflow_PostFMRIPREP.py'
data_dir = Path('/projects/osl/xiy/data/210817SAGE2Data')

task_list = ['reward']
run_list = ['01 02 03 04 05 06', '01 02 03 04 05 06']
bold_space = 'MNI152NLin2009cAsym'

output_dir = deriv_dir.joinpath('preproc_variant-smooth6')
out_file = submit_dir.joinpath(f'preproc_post-fmriprep_space-{bold_space}_variant-smooth6.txt')

op_string = ''
for bold_id, subj_id in product(zip(task_list, run_list), subj_list):

    task_id = bold_id[0]
    run_id = bold_id[1]
    work_dir = submit_dir.joinpath('works', (f'preproc_post-fmriprep_task-{task_id}_'
                                           f'space-{bold_space}_variant-smooth6'))

    op_string += (
        f'python3 {workflow} '
        f'{data_dir} {output_dir} participant '
        f'--participant_label {subj_id} --task-id {task_id} --run-id {run_id} '
        f'--bold-space {bold_space} --brainmask '
        f'fmriprep '
        '--repetition-time 2.0 --smooth-fwhm 6'
        f'--n_procs 6 --work-dir {work_dir}\n')
    out_file.write_text(op_string)


In [8]:
# subregion
output_dir = deriv_dir / 'hippocampus_subregions'
out_file = submit_dir / 'preproc_hippocampus-segmentation.txt'
op_string = ''
for subj_id in subj_list:
    
    # Excludes sub-28 due to lack of T2w image
#     if subj_id == '28':
#         continue
    
    op_string += (
        f'singularity exec -e --bind /projects/kuhl_lab/yzhao17:/projects/kuhl_lab/yzhao17 /projects/kuhl_lab/yzhao17/Image/Valhalla.simg '
        '/opt/ashs/bin/ashs_main.sh '
        f'-I sub-{subj_id} '
        '-a /opt/resources/ashs_atlas/ashs_atlas_upennpmc_20170810 '
        f'-g {deriv_dir}/fmriprep/sub-{subj_id}/anat/sub-{subj_id}_desc-preproc_T1w.nii.gz '
        f'-f {bids_dir}/sub-{subj_id}/anat/sub-{subj_id}_T2w.nii.gz '
        f'-w {output_dir}/sub-{subj_id} -T\n')
    out_file.write_text(op_string)

In [8]:
# create roi
out_file = submit_dir / 'create_de_roi.txt'
op_string = ''
for subj_id in subj_list:
    op_string += (
        'singularity run -e --bind /projects/kuhl_lab/yzhao17:/projects/kuhl_lab/yzhao17 /projects/kuhl_lab/yzhao17/Image/Valhalla.simg '
        f'python3 /projects/kuhl_lab/yzhao17/DIIN/scripts/roi/create_roi_destrieux.py '
        f'--participant_label {subj_id}\n')
    out_file.write_text(op_string)